In [1]:
!pip install pymysql

In [2]:
!pip install pykrx

    high ,#고가
    low ,#저가
    end_close #종가
    trading_volume #거래량
    transaction_amount  #거래대금
    end_rate_change #등락율
    institutional_total #기관 합계
    other_corporations #기타 법인
    individual #개인
    foreigner_total # 외국인 합계
    short_selling ,#공매도
    short_buying # 공매도 매수
    short_importance #공매도 비중

In [1]:
import joblib
import pymysql
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import platform
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from pykrx import stock

In [3]:
conn = pymysql.connect(host='localhost', user='root', password='a026195', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)

# 총합 정확도 예측을 위한 변수
total = 0

#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 100

try:
    # for문에 
    for i in range(0,stock_n):

        curs = conn.cursor()
#         sql = "SELECT * FROM StockData WHERE stock_code ='{}'".format(str(stock_code_list['stock_code'][i]))
        
        sql = "SELECT * FROM StockData WHERE stock_code ='A004020'"


        # 코드 앞에서 A 지우고 주식 이름 불러오기
#         name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))

        name = stock.get_market_ticker_name(str('004020'))
        curs.execute(sql)
        data = curs.fetchall()
        
        df = pd.DataFrame(data)
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)
    
        # X1 일 뒤 X2일 평균 종가
        X1 = 10
        X2 = 5
        df['X1일 뒤 종가'] = df['end_close'].shift(-X1)
        df['X1일 뒤 X2일 평균 종가'] = 0
        for i in range(0, df.shape[0]-X2):
            for j in range(0,X2):
                df['X1일 뒤 X2일 평균 종가'][i] = df['X1일 뒤 X2일 평균 종가'][i] + df['X1일 뒤 종가'][i+j]
            
        df['X1일 뒤 X2일 평균 종가'] = df['X1일 뒤 X2일 평균 종가']/X2
    
        df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가'] - df['end_close']
    
        # 분류 모델 쓸 때 0,1로 라벨링 하기
        df['X1일 뒤 X2일 평균 종가 변화량'] = df['X1일 뒤 X2일 평균 종가 변화량'].apply(lambda x: 1 if x>0 else 0)

        df.dropna(inplace=True)
        sql2 = "SELECT * FROM MacroeconomicIndicators"
        curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']

        
        target = ['X1일 뒤 X2일 평균 종가 변화량']
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi','inflation', 'treasury_bonds', 
                 'tightening','normality', 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        
        train_x, test_x, train_y, test_y = train_test_split(df_m[f_col], df_m[target], test_size=0.15,shuffle=False)
        
        #xgbboost
        xgb = XGBClassifier(n_estimators=200)
        xgb.fit(train_x, train_y)
        y_pred = xgb.predict(test_x)
        train_pred = xgb.predict(train_x)
        
        total = total + f1_score(test_y, y_pred, average='macro')
        
        joblib.dump(xgb, 'model/'+name+'_model.pkl')

        print("AI 예측 정확도 - " + name + ":",f1_score(test_y, y_pred, average='macro'))
    
    print("종목 총합 정확도 : ",(total/stock_n))

finally:
    conn.close()

AI 예측 정확도 - 현대제철: 0.5266531241244046
종목 총합 정확도 :  0.5266531241244046


In [4]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(xgb, scoring = "f1").fit(test_x, test_y)
eli5.show_weights(perm, top = 20, feature_names = test_x.columns.tolist())

Weight,Feature
0.0589 ± 0.0403,learning_result
0.0328 ± 0.0575,60일 평균 종가 대비 당일 종가 비율
0.0266 ± 0.0302,코스피지수5일평균종가대비당일종가비율
0.0200 ± 0.0000,코스피 20일 평균 종가
0.0165 ± 0.0211,코스피지수60일평균종가대비당일종가비율
0.0115 ± 0.0086,전일종가대비 당일 시가비율
0.0104 ± 0.0140,20일평균거래량대비당일거래량
0.0100 ± 0.0126,chairman
0.0084 ± 0.0085,1yd_trading
0.0068 ± 0.0167,당일종가 대비 당일 저가 비율


In [6]:
conn = pymysql.connect(host='localhost', user='root', password='a026195', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
sql = "SELECT * FROM StockData WHERE stock_code ='A329180'"
# 코드 앞에서 A 지우고 주식 이름 불러오기
name = stock.get_market_ticker_name(str('329180'))
curs.execute(sql)
data = curs.fetchall()
        
df = pd.DataFrame(data)

df

,id,stock_code,data_time,start_open,high,low,end_close,trading_volume,transaction_amount,end_rate_change,institutional_total,other_corporations,individual,foreigner_total,short_selling,short_buying,short_importance
0,74376,A329180,2021-09-17,111000.0,135000.0,91000.0,111500.0,17373500.0,1.955250e+12,0.45,1.475130e+11,-3.325930e+09,4.208510e+10,-1.862720e+11,1000.0,17373500.0,0.01
1,74377,A329180,2021-09-23,112000.0,113000.0,99000.0,99100.0,2914290.0,3.021960e+11,-11.12,9.212490e+09,-7.167420e+08,3.185690e+10,-4.035270e+10,275.0,2914290.0,0.01
2,74378,A329180,2021-09-24,100000.0,102000.0,95500.0,99200.0,2262590.0,2.236640e+11,0.10,1.858430e+10,-5.591730e+07,-7.937340e+09,-1.059110e+10,867.0,2262590.0,0.04
3,74379,A329180,2021-09-27,98300.0,99400.0,96500.0,97000.0,672212.0,6.592860e+10,-2.22,9.807240e+09,-1.990470e+07,-1.073040e+10,9.431150e+08,898.0,672212.0,0.13
4,74380,A329180,2021-09-28,98000.0,103000.0,97000.0,100000.0,1037430.0,1.043380e+11,3.09,1.568760e+10,3.919060e+08,-1.605430e+10,-2.520530e+07,145.0,1037430.0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,74683,A329180,2022-12-19,118500.0,120000.0,117500.0,119000.0,64634.0,7.698210e+09,0.42,2.948160e+09,-3.342850e+07,-2.158720e+09,-7.560040e+08,13328.0,64634.0,20.62
308,74684,A329180,2022-12-20,120000.0,120000.0,116500.0,117500.0,75541.0,8.933960e+09,-1.26,5.804340e+08,-2.466650e+07,-7.497610e+08,1.939940e+08,10035.0,75541.0,13.28
309,74685,A329180,2022-12-21,118500.0,122000.0,117500.0,121000.0,224620.0,2.713680e+10,2.98,6.715440e+09,-2.565630e+08,-1.393830e+10,7.479460e+09,4188.0,224620.0,1.86
310,74686,A329180,2022-12-22,121500.0,123500.0,121000.0,122000.0,155591.0,1.901920e+10,0.83,5.597530e+09,-2.414300e+08,-3.874940e+09,-1.481170e+09,14046.0,155591.0,9.03


In [48]:
conn = pymysql.connect(host='localhost', user='root', password='a026195', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)
stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)


#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 1

try:
    # for문에 
    for i in range(0,stock_n):

        curs = conn.cursor()
        sql = "SELECT * FROM StockData WHERE (data_time BETWEEN '2021-01-01' AND '2022-12-09') AND (stock_code ='{}')".format(str(stock_code_list['stock_code'][i]))
        # 코드 앞에서 A 지우고 주식 이름 불러오기
        name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
        curs.execute(sql)
        data = curs.fetchall()
        
        df = pd.DataFrame(data)
        
        df = df[-300:]
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)

        df.dropna(inplace=True)
        
        sql2 = "SELECT * FROM MacroeconomicIndicators WHERE (date_time BETWEEN '2021-01-01' AND '2022-12-09')"
        curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)
        
        df2 = df2[-300:]

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']
        
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi', 
                 'inflation', 'treasury_bonds', 'tightening','normality', 
                 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        
        test_x = df_m[f_col]
        
        model = joblib.load('model/'+name+'_model.pkl')

        y_pred = model.predict(test_x)

        print()
        print(name + "의 " + str(df_m['data_time'][0])+" 예측 결과 = ",y_pred)
finally:
    conn.close()


삼성전자의 2022-03-22 예측 결과 =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
